In [11]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import numpy as np
import pandas as pd
import pickle, os, argparse
import multiprocessing as mp
import h5py
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from controls_main import RunInfo
from rowwise_neuron_curves_controls import compute_metrics, lstring, get_binidx, get_centers, unit_vector, \
    angle_xaxis, get_polar, linreg, feature_set, X_data

# GLOBAL PARS
t_stride = 2
ntime=320
metrics = ['RMSE', 'r2', 'PCC']
nmetrics = len(metrics)

specify which model, run config, etc

In [12]:
modelinfo = dict({'type': 'S',
            'base': 'spatial_temporal_4_8-16-16-32_64-64-64-64_5272',
            'name': 'spatial_temporal_4_8-16-16-32_64-64-64-64_5272_1',
            'shortbase': 'Spatial-Temporal',
            'shortname': 'Spatial-Temporal_1',
            'nlayers': 8,
            'max_act': 14,
            'control': False,
            'cmap': 'Blues_r',
            'color': 'C0',
            'control_cmap': 'Purples_r'})

runinfo = RunInfo({'expid': 102, #internal experiment id
                   'datafraction': 0.5,
                   'randomseed': 2000,
                   'randomseed_traintest': 42,
                   'dirr2threshold': 0.2,
                   'verbose': 0,
                   'model_experiment_id': 4, #as per Pranav's model generation
                   'basefolder' : '/home/kai/Dropbox/DeepDrawData/analysis-data/', #point to analysis-data folder
                       #in DeepDrawData folder from Dropbox, include trailing slash
                   'orientation' : 'hor',
                   'height' : 'all'
            })

specify which layer and feature set you want to compute the tuning curves for

In [59]:
ilayer = 2
fset='dir' #options: ['dir', 'vel', 'dirvel', 'acc', 'labels', 'ee', 'eepolar']

get kinematic data

In [60]:
modelname = modelinfo['name']
nlayers = modelinfo['nlayers']
base = modelinfo['base']
layer = lstring(ilayer)
expid = runinfo['expid']

In [61]:
X, xyplmvt = X_data(fset, runinfo, datafolder=runinfo.datafolder(modelinfo))

/mnt/data/random_controls/exp102/data/spatial_temporal_4_8-16-16-32_64-64-64-64_5272/spatial_temporal_4_8-16-16-32_64-64-64-64_5272_1
using alternate method for accessing kinvars files by directly accessing needed arrays (pandas causes error)


tune layer

In [62]:
lo = pickle.load(open(os.path.join(runinfo.datafolder(modelinfo), layer + '.pkl'), 'rb'))
lo = lo[xyplmvt]

centers = get_centers(lo.shape[2])

if (fset == 'vel' or fset == 'acc' or fset == 'eepolar' or fset == 'ang' or fset=='angvel' or fset=='ee'):
    nmods = 4
    nmets = 6
elif (fset == 'labels'):
    nmods = 1
    nmets = 1

In [63]:
Y = lo

# Resize Y so that feature maps are appended as new rows in first feature map
Y = Y.swapaxes(1,2).reshape((Y.shape[0], Y.shape[2], -1)).swapaxes(1,2)

In [64]:
trainevals =  np.empty((Y.shape[1], nmods, nmets))
testevals = np.empty((Y.shape[1], nmods, nmets))
    #Axis 0-k: lo dims except time
    #Axis k+1: (0) LinReg (1) Dir Tuning (2) Vel Tuning (3) Dir + Vel Tuning
    #Axis k+2: (0) RMSE (1) r2 (2) PCC (3-5) Reg Coeffs

In [65]:
# %% DIRECTION AND VELOCITY TUNING 
def tune_row_vel(X, Y, isPolar = True):
    ''' Fit tuning curves for a single neuron / unit ("row") for velocity inputs (and similar kinematic features) for four different kinds of models
        1. simple linear regression over input features of X
        2. Directional tuning curves
        3. Velocity / Speed Dependence
        4. Direction x Velocity tuning curve
    
    Arguments
    ---------
    X : np.array [nr of samples, 2] 
    Y : np.array [nr of samples]
    row : int, row index
    isPolar : bool, is result already boolean?
    
    Returns
    -------
    tuple of 
        row : int, row index
        rowtraineval : np.array [4, 6] for four different model types. Cols 0-2: Metrics from compute_metrics, Cols 3-5: Linear Regression coeffs
        rowtesteval : np.array [4, 6] for four different model types. Cols 0-2: Metrics from compute_metrics, Cols 3-5: Linear regression coeffs
    '''
    
    rowtraineval = np.zeros((4,6))
    rowtesteval = np.zeros((4,6))
    #Axis 0: (0) training set (1) test set
    #test set els 4-6: linear regression coeffs
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

    ##BASELINE LINREG MODEL
    print("Baseline Linear Regression Model:")
    #Xtform_train = np.c_[X_train[:,0], X_train[:,1], np.ones_like(X_train[:,0])]
    #Xtform_test = np.c_[X_test[:,0], X_test[:,1], np.ones_like(X_test[:,0])]
    Xtform_train, Xtform_test, Ytform_train, Ytform_test = feature_set(
            (X_train, np.ones_like(X_train[:,0])),
            (X_test, np.ones_like(X_test[:,0])),
            Y_train,
            Y_test
            )

    rowtraineval[0], rowtesteval[0] = linreg(Xtform_train, Xtform_test, Ytform_train, Ytform_test)
    
    #Change to polar coordinates
    if(not isPolar):
        print("converting to polar...")
        X_train = get_polar(X_train)
        X_test = get_polar(X_test)
    
    ##DIR DEP
    print("Directional Dependence:")
    #Xtform_train = np.c_[np.cos(X_train[:,1]), np.sin(X_train[:,1]), np.ones_like(X_train[:,1])]
    #Xtform_test = np.c_[np.cos(X_test[:,1]), np.sin(X_test[:,1]), np.ones_like(X_test[:,1])]
    Xtform_train, Xtform_test, Ytform_train, Ytform_test = feature_set(
        (np.cos(X_train[:,1]), np.sin(X_train[:,1]), np.ones_like(X_train[:,0])),
        (np.cos(X_test[:,1]), np.sin(X_test[:,1]), np.ones_like(X_test[:,0])),
        Y_train,
        Y_test
        )

    rowtraineval[1], rowtesteval[1] = linreg(Xtform_train, Xtform_test, Ytform_train, Ytform_test)
    
    ##VEL DEP
    
    #Xtform_train = np.c_[X_train[:,0], np.ones_like(X_train[:,0])]
    #Xtform_test = np.c_[X_test[:,0], np.ones_like(X_test[:,0])]
    print("Velocity Dependence:")
    Xtform_train, Xtform_test, Ytform_train, Ytform_test = feature_set(
        (X_train[:,0], np.ones_like(X_train[:,0])),
        (X_test[:,0], np.ones_like(X_test[:,0])),
        Y_train,
        Y_test
        )
    rowtraineval[2], rowtesteval[2] = linreg(Xtform_train, Xtform_test, Ytform_train, Ytform_test)
    
    ##DIR PLUS VEL DEP
    #Xtform_train = np.c_[X_train[:,0] * np.cos(X_train[:,1]), X_train[:,0] * np.sin(X_train[:,1]), np.ones_like(X_train[:,0])]
    #Xtform_test = np.c_[X_test[:,0] * np.cos(X_test[:,1]), X_test[:,0] * np.sin(X_test[:,1]), np.ones_like(X_test[:,0])]
    print("Direction + Velocity Dependence:")   
    Xtform_train, Xtform_test, Ytform_train, Ytform_test = feature_set(
        (X_train[:,0] * np.cos(X_train[:,1]), X_train[:,0] * np.sin(X_train[:,1]), np.ones_like(X_train[:,0])),
        (X_test[:,0] * np.cos(X_test[:,1]), X_test[:,0] * np.sin(X_test[:,1]), np.ones_like(X_test[:,0])),
        Y_train,
        Y_test
        )
    
    rowtraineval[3], rowtesteval[3] = linreg(Xtform_train, Xtform_test, Ytform_train, Ytform_test)
        
    return (rowtraineval, rowtesteval)

# %% LABEL SPECIFICITY
    
def tune_row_label(X, Y):
    """Calculate label specificity based on ROC AUC score based on Linear SVMs 
    trained according to a OneVsRest strategy
    
    Arguments
    ----------
    X: labels
    Y: node activitiy for each sample
    
    Returns
    ----------
    np.array([(nodetraineval-0.5)*2]), np.array([(nodetesteval-0.5)*2]): 
        np.array of floats [1,], return normalized ROC AUC score for given node
    """
    
    X = label_binarize(X, np.unique(X))
    Y = Y.reshape(-1,1)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)
    
    try:
        svm = OneVsRestClassifier(LinearSVC(max_iter=10, verbose=0))
        svm.fit(Y_train, X_train)
        
        nodetraineval = roc_auc_score(X_train, svm.decision_function(Y_train))
        try:
            nodetesteval = roc_auc_score(X_test, svm.decision_function(Y_test))
        except ValueError as err:
            print("test evaluation failed. %s" %err) 
            nodetesteval = 0.5
            
    except ValueError as err:
         print("fitting SVM failed. %s" %err) 
         nodetraineval = 0.5
        
    print("Train eval: %s" %str(nodetraineval))
    print("Test eval: %s" %str(nodetesteval))
    
    return np.array([(nodetraineval-0.5)*2]), np.array([(nodetesteval-0.5)*2])


In [66]:
for irow in range(Y.shape[1]):
    print("Row: ", irow)

    if(len(X.shape) > 1):
        x = X[..., centers]
    else:
        x = X

    y = Y[:,irow]

    ##RESHAPE FOR LINEAR REGRESSION

    if len(x.shape) > 1: ##FOR TIME-BASED DATA ( 2 COMPS PER TIMESPACE IS USE CASE)

        tcoff = sum(np.where(centers <= 32, True, False))
        x = x[...,tcoff:ntime-tcoff]
        y = y[:,tcoff:ntime-tcoff]
        x = x.swapaxes(1,2).reshape((-1,2))
    elif fset == 'labels':
        temp = np.ones_like(y)
        x = (temp.swapaxes(0,1) * x).swapaxes(0,1)
        x = x.reshape((-1,))
    y = y.reshape((-1,))

    if fset == 'acc' or fset == 'vel' or fset == 'eepolar' or fset == 'ang' or fset=='angvel' :
        trainevals[irow], testevals[irow] = tune_row_vel(x,y, True)
    elif fset == 'ee':
        trainevals[irow], testevals[irow] = tune_row_vel(x,y, False)
        #sys.stdout.flush()
    elif fset == 'labels':
        trainevals[irow], testevals[irow] = tune_row_label(x,y)

Row:  0
Row:  1
Row:  2
Row:  3
Row:  4
Row:  5
Row:  6
Row:  7
Row:  8
Row:  9
Row:  10
Row:  11
Row:  12
Row:  13
Row:  14
Row:  15
Row:  16
Row:  17
Row:  18
Row:  19
Row:  20
Row:  21
Row:  22
Row:  23
Row:  24
Row:  25
Row:  26
Row:  27
Row:  28
Row:  29
Row:  30
Row:  31
Row:  32
Row:  33
Row:  34
Row:  35
Row:  36
Row:  37
Row:  38
Row:  39
Row:  40
Row:  41
Row:  42
Row:  43
Row:  44
Row:  45
Row:  46
Row:  47
Row:  48
Row:  49
Row:  50
Row:  51
Row:  52
Row:  53
Row:  54
Row:  55
Row:  56
Row:  57
Row:  58
Row:  59
Row:  60
Row:  61
Row:  62
Row:  63


print some aggregate statistics for illustration

In [67]:
def print_statistics(stats):
    """print some aggregate statistics about the tuning strengths
    
    Arguments
    ---------
    stats : np.array of floats
    """
    
    stats = stats.flatten()
    print('Min: %.4f' %stats.min())
    print('Max: %.4f' %stats.max())
    print('Mean: %.4f' %stats.mean())
    print('25pc Quantile: %.4f' %np.quantile(stats, 0.25))
    print('Median: %.4f' %np.quantile(stats, 0.5))
    print('75pc Quantile: %.4f' %np.quantile(stats, 0.75))

In [69]:
print("%s Layer %d: " %(modelinfo['name'], ilayer + 1))
if fset != 'labels':
    print("\nRMSE: ")
    print_statistics(testevals[:,0])

    print("\n\nR2: ")
    print_statistics(testevals[:,1])

    print("\n\nPCC: ")
    print_statistics(testevals[:,2])
else:
    print('\nNormalized AUC ROC Score: ')
    print_statistics(testevals[:,0])

spatial_temporal_4_8-16-16-32_64-64-64-64_5272_1 Layer 3: 

RMSE: 
Min: -0.0085
Max: 0.0389
Mean: 0.0097
25pc Quantile: 0.0046
Median: 0.0093
75pc Quantile: 0.0127


R2: 


IndexError: index 1 is out of bounds for axis 1 with size 1